In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
tf.gfile = tf.io.gfile
import tensorflow_hub as hub
from bert import tokenization
from sklearn.utils import resample
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tensorflow import keras
from official.nlp import optimization
import os
import tensorflow_text

create model

In [ ]:
preprocessed_data = pd.read_csv("preprocess_data.csv")

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(preprocessed_data["Sentence"],preprocessed_data["Disease"], test_size=0.1,stratify=None)

In [ ]:
train_labels = keras.utils.to_categorical(y_train.values, num_classes=41)
test_labels = keras.utils.to_categorical(y_test.values, num_classes=41)

In [ ]:
def model(metrics):

    text_input = tf.keras.layers.Input(shape=(), dtype=tf.string)
    preprocessor = hub.KerasLayer(
        "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
    
    encoder_inputs = preprocessor(text_input)
    encoder = hub.KerasLayer(
        "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4",
        trainable=True)
    
    outputs = encoder(encoder_inputs)
    pooled_output = outputs["pooled_output"] 
    sequence_output = outputs["sequence_output"]

    clf_output = sequence_output[:, 0, :]
    net = tf.keras.layers.Dense(32, activation='relu')(clf_output)
    net = tf.keras.layers.Dropout(0.2)(net)
    out = tf.keras.layers.Dense(41, activation='softmax')(net)
    
    model = tf.keras.models.Model(inputs=text_input, outputs=out)
    model.compile(tf.keras.optimizers.Adam(lr=1e-5), loss='categorical_crossentropy', metrics=metrics)
    
    return model

In [ ]:
METRICS = [
    tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]

bert_model = model(METRICS)
bert_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None,)]            0           []                               
                                                                                                  
 keras_layer_4 (KerasLayer)     {'input_mask': (Non  0           ['input_3[0][0]']                
                                e, 128),                                                          
                                 'input_word_ids':                                                
                                (None, 128),                                                      
                                 'input_type_ids':                                                
                                (None, 128)}                                                

/usr/local/lib/python3.7/dist-packages/keras/optimizers/optimizer_v2/adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Train Model

In [ ]:
train_history = bert_model.fit(
    X_train, train_labels, 
    validation_split=0.2,
    epochs=3,
    verbose=1
)

Epoch 1/3
3632/3632 [==============================] - 3401s 933ms/step - loss: 0.4744 - accuracy: 0.9947 - precision: 0.9593 - recall: 0.8154 - val_loss: 0.1543 - val_accuracy: 0.9978 - val_precision: 0.9650 - val_recall: 0.9423
Epoch 2/3
3632/3632 [==============================] - 3382s 931ms/step - loss: 0.2025 - accuracy: 0.9973 - precision: 0.9658 - recall: 0.9227 - val_loss: 0.1396 - val_accuracy: 0.9978 - val_precision: 0.9660 - val_recall: 0.9445
Epoch 3/3
3632/3632 [==============================] - 3371s 928ms/step - loss: 0.1672 - accuracy: 0.9976 - precision: 0.9686 - recall: 0.9339 - val_loss: 0.1223 - val_accuracy: 0.9980 - val_precision: 0.9668 - val_recall: 0.9486


In [ ]:
bert_model.save_weights('bert_weights')

In [ ]:
saver = tf.train.Saver(tf())